In [ ]:
import numpy as np
import numpy.typing as npt
from matplotlib import pyplot as plt, animation as animation, ticker as mticker
from matplotlib.pyplot import cm
import matplotlib as mpl
plt.rcParams.update({'font.size': 10})
from matplotlib.patches import Ellipse, Circle
import matplotlib.transforms as transforms
import pandas as pd
import scipy as sp
import astropy as ap
from tqdm.notebook import tqdm
from sklearn.cluster import DBSCAN
from scipy.interpolate import splprep, splev
from scipy.stats import gaussian_kde


from fireworks.particles import Particles
import fireworks.nbodylib.nunits as NU
import fireworks.nbodylib.potentials as fnp
import fireworks.nbodylib.dynamics as fnd
import fireworks.nbodylib.integrators as fni

In [ ]:
def cartesian2spherical(rectangular_coordinates: list, out_type:str='rad'):
    """
    Converts Cartesian coordinates to spherical coordinates.
    Parameters
    ----------
    rectangular_coordinates : list
        A list of coordinates of the point in Cartesian space.
    out_type : str, optional
        Whether the theta and phi arguments in output should be in degree or
        radian units. The default is 'rad'.
    Returns
    -------
    rho, theta, phi
    """
    # initialize the coordinates
    x, y, z = rectangular_coordinates
    # calculate rho
    rho = np.sqrt(x**2 + y**2 + z**2)
    # calculate theta
    theta = np.arccos(z / rho)
    # calculate phi, the reason for using arctan2 is that the range of theta
    # parameter is -180 < theta < 180.
    phi = np.arctan2(y, x)
    if out_type == 'deg':
        theta, phi = np.degrees([theta, phi])
    return rho, theta, phi

In [ ]:
def cyl_coord(pos, vel):
    rho = np.sqrt(pos[..., 0]**2 + pos[..., 1]**2)
    phi = np.arctan2(pos[..., 1], pos[..., 0])
    phi = np.where(phi < 0, phi + 2*np.pi, phi)

    vel_r = vel[...,0]*np.cos(phi) + vel[...,1]*np.sin(phi)
    vel_phi = -vel[...,0]*np.sin(phi) + vel[...,1]*np.cos(phi)
    vel_z = vel[...,2]
    
    return rho, phi, vel_r, vel_phi, vel_z


In [ ]:
#single run 
N = 5000
initial_position = 10    #pos del 
M_G = 1e3    # mass of the galaxy
e=0.5
t=0.01
semi_major_axis = initial_position/(1-e)
data = np.load(f'Orbit/Galactic_FoR/{N}_InitialPos_{initial_position}_e_{e}_t_{t}.npy')
position = data[:, :, :3]
velocity = data[:, :, 3:6]
mass = data[:, :, 6]
energy = data[:, 0, 7]
potential_energy = data[:, :, 8]

# # data from mmultiple run 
# N = 1000
# initial_position = 30    #pos del 
# M_G = 1e3    # mass of the galaxy
# e=0.9
# t=0.01
# semi_major_axis = np.sqrt( initial_position/(1-e**2)  )
# data = np.load('Orbit/Galactic_FoR/Simulations_multiple_orbits/sim_e05_e09_ip20.npy')
# print(data.shape)
# data = data[1, :, :, :]
# position = data[:, :, :3]
# velocity = data[:, :, 3:6]
# mass = np.ones_like(position)[:, :, 0]


# Tidal radius and iterative calculation of the center of mass 

In [ ]:
def flag_cluster(position: npt.NDArray[np.float64], velocity: npt.NDArray[np.float64], mass: npt.NDArray[np.float64], N: int, M_G: float):
    
    """
    Function to find the stars that are bound to the cluster. Create a flag array with the same length as the number of stars
    in the cluster. The flag array is set to True for the stars that are bound to the cluster and False for the stars that are
    unbound. The function returns the flag array, the center of mass of the cluster and the tidal radius of the cluster.

    Parameters
    ----------
    position : npt.NDArray[np.float64]
        Array with the positions of the stars at each snapshot

    velocity : npt.NDArray[np.float64]
        Array with the velocities of the stars at each snapshot

    mass : npt.NDArray[np.float64]
        Array with the mass of the stars at each snapshot

    N : int
        Number of stars in the cluster

    M_G : float
        Mass of the galaxy
    """

    flag_history = []
    com_history = []
    vel_com_history = []
    tidal2_history = []

    flag = np.ones(N, dtype=bool)
    for snapshot in tqdm(range(len(position))):
        pos_snap, vel_snap = position[snapshot], velocity[snapshot]

        current_flag = np.zeros(N, dtype=bool)
        k = 0
        while flag.sum() - current_flag.sum() != 0:
            if k>100:
                break
            current_flag = flag.copy()
            if mass[snapshot][flag].sum() == 0:
                pos_com_snap = np.average(pos_snap[flag], axis=0)
                vel_com_snap = np.average(vel_snap[flag], axis=0)
            else:
                pos_com_snap = np.average(pos_snap[flag], axis=0, weights=mass[snapshot][flag])
                vel_com_snap = np.average(vel_snap[flag], axis=0, weights=mass[snapshot][flag])


            # Calculate distances to the center of mass
            distances_to_com = np.sqrt(np.sum((pos_snap - pos_com_snap)**2, axis=1))

            # Calculate tidal radius
            r_t = np.linalg.norm(pos_com_snap) * (np.sum(mass[snapshot][flag]) / M_G) ** (1 / 3)

            # Update flag based on tidal radius
            flag = distances_to_com < 2*r_t

        flag_history.append(flag)
        com_history.append(pos_com_snap)
        vel_com_history.append(vel_com_snap)
        tidal2_history.append(r_t)

    return flag_history, com_history, vel_com_history, tidal2_history


In [ ]:
flag_history, com_history, vel_com_history, tidal2_history = flag_cluster(position, velocity, mass, N, M_G)

# Distance between true CoM and iterative CoM in tidal radius

In [ ]:
fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot()
ax.plot(np.linalg.norm(np.array(com_history) - position[:].mean(axis=1), axis=1))
ax.set_xlabel('Tstep', fontsize=15)
ax.set_ylabel(r'$\Delta (r_{CoM} - r_{CoM, tidal})$ [Henon units]',fontsize=15)
fig.savefig(fname=f'/ca23/ext_volume/pod_compastro23/Project/Images/{e}/Delta_r_tidal_{N}_InitialPos_{initial_position}_e_{e}.png', bbox_inches='tight')

# Analysis considering only the stars inside the tidal radius

### %_mass_radius

In [ ]:
position_comFoR = np.zeros_like(position)
for i in range(len(position)):
    position_comFoR[i] = position[i] - np.array(com_history[i])


tenperc_mass_radius = []
half_mass_radius = []
ninethyperc_mass_radius = []

for time in range(int(len(position_comFoR))):
    position_comFoR_flagged = position_comFoR[time][flag_history[time]]
    N_flagged = len(position_comFoR_flagged)
    current_radius = np.linalg.norm(position_comFoR[time], axis=1)
    sorted_radius = sorted(current_radius)
    tenperc_mass_radius.append(sorted_radius[int(N_flagged/10)])
    half_mass_radius.append(sorted_radius[int(N_flagged/2)])
    ninethyperc_mass_radius.append(sorted_radius[int(9*N_flagged/10)])
    
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot()
ax.plot(tenperc_mass_radius, label='10% mass radius')
ax.plot(half_mass_radius, label='half mass radius')
ax.plot(ninethyperc_mass_radius, label='90% mass radius')

ax.legend()
ax.grid(linestyle='dotted')
ax.set_xlabel('Time steps')
f = lambda x: x*0.01
g = lambda x: x/0.01
ax2 = ax.secondary_xaxis("top", functions=(f, g))
ax2.set_xlabel("Time [Henon unit]")
ax.set_ylabel(r'$|r - r_{com}|$ [Henon unit]') 
ax.set_title('Radii containing a given amount of mass')
ax.set_ylim(0.0, 1.6)
fig.savefig(fname=f'/ca23/ext_volume/pod_compastro23/Project/Images/{e}/r_perc_mass_{N}_InitialPos_{initial_position}_e_{e}.png', bbox_inches='tight')

# Mass loss

In [ ]:
flag_history = np.array(flag_history)
mass_loss = abs(flag_history[1:, :].sum(axis=1) - flag_history[:-1, :].sum(axis=1))
mass_loss_cum = np.cumsum(mass_loss)

fig = plt.figure(figsize=(20, 5))
ax = fig.add_subplot(121)
ax.plot( mass_loss)
ax.set_xlabel('Time', fontsize=15)
ax.set_ylabel('Mass Loss [Henon unit]', fontsize=15)
ax.set_xlabel('time steps', fontsize=15)
f = lambda x: x*0.01
g = lambda x: x/0.01
ax2 = ax.secondary_xaxis("top", functions=(f, g))
ax2.set_xlabel("Time [Henon unit]",fontsize=15)

ax = fig.add_subplot(122)
ax.plot(mass_loss_cum)
ax.set_ylabel('Cumulative Mass Loss [Henon unit]', fontsize=15)
ax.set_xlabel('time steps',fontsize=15)
ax2 = ax.secondary_xaxis("top", functions=(f, g))
ax2.set_xlabel("Time [Henon unit]",fontsize=15)
fig.savefig(fname=f'/ca23/ext_volume/pod_compastro23/Project/Images/{e}/MassLoss_{N}_InitialPos_{initial_position}_e_{e}.png', bbox_inches='tight')

### Density profile n_bins is the number of shell considered

In [ ]:
n_bins=5

fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot()
time_to_plot = [0, 100, 150, 200, 250, 400]
color = cm.turbo(np.linspace(0, 1, len(time_to_plot)))

for k, c in enumerate(color):
    time = time_to_plot[k]
    position_comFoR_flagged = position_comFoR[time][flag_history[time]]
    radius_snapshot = np.linalg.norm(position_comFoR_flagged, axis=1)
    counts, bin_edges = np.histogram(radius_snapshot, bins=n_bins)
    Vol_shell = 4/3 *np.pi* (bin_edges[1:]**3 - bin_edges[:-1]**3)
    density = counts / Vol_shell
    radius_shell = 1/2 * (bin_edges[1:] + bin_edges[:-1])
    
    ax.plot(radius_shell, density, color=c, label=f'Tstep:{time}', linewidth=2)
    ax.set_xlabel(r'|$r - r_{com}|_{shell}$ [Henon units]', fontsize=15)
    ax.set_ylabel(r'$\rho [\frac{Mass}{Vol_{shell}}]$ [Henon units]', fontsize=15)
    ax.legend(loc='upper right', fontsize=10)
    ax.set_title(f'Density profile across time', fontsize=15)
fig.savefig(fname=f'/ca23/ext_volume/pod_compastro23/Project/Images/{e}/Density_profile_{N}_InitialPos_{initial_position}_e_{e}.png', bbox_inches='tight')

# Velocity dispersion in Shell

In [ ]:
fig = plt.figure(figsize=(22, 5))
time_to_plot = [0, 100, 150, 200, 250, 400]
color = cm.turbo(np.linspace(0, 1, len(time_to_plot)))
n_bins=5

ax = fig.add_subplot(1,3,1)
for k, c in enumerate(color):
    time = time_to_plot[k]
    position_comFoR_flagged = position_comFoR[time][flag_history[time]]
    radius_snapshot = np.linalg.norm(position_comFoR_flagged, axis=1)
    counts, bin_edges = np.histogram(radius_snapshot, bins=n_bins)
    radius_shell = 1/2 * (bin_edges[1:] + bin_edges[:-1]) #needed to plot
    
    bins_index = np.digitize(radius_snapshot, bins=bin_edges) #return the index of the bin for each radius
    
    vel_snapshot = velocity[time][flag_history[time]]-np.linalg.norm(vel_com_history[time]) #- vel_com_history[time]
    vel_modul_snapshot = np.linalg.norm(vel_snapshot, axis=1)
    
    vel_modul_snaphost_binned = [vel_modul_snapshot[bins_index==i].tolist() for i in range(1, len(bin_edges)) ] #binning the velocity as the radius
    
    velocity_dispersion = []
    for i in range(len(bin_edges)-1):
        velocity_dispersion.append(np.array(vel_modul_snaphost_binned[i]).std())

    ax.plot(radius_shell, np.array(velocity_dispersion), label=f'Tstep: {time}', color=c)
    ax.set_xlabel(r'|$r - r_{com}|_{shell}$ [Henon units]', fontsize=15)
    ax.set_ylabel(r'$\sigma_{{(|v- v_{com}|)}_{shell}}$ [Henon units]',fontsize=15 )
    ax.set_title('Velocity modulus dispersion profile across time', fontsize=15)
    ax.set_ylim(-0.3, 5)
    ax.legend()

ax = fig.add_subplot(1,3,2)
for k, c in enumerate(color):
    time = time_to_plot[k]
    position_comFoR_flagged = position_comFoR[time][flag_history[time]]
    radius_snapshot = np.linalg.norm(position_comFoR_flagged, axis=1)
    counts, bin_edges = np.histogram(radius_snapshot, bins=n_bins)
    radius_shell = 1/2 * (bin_edges[1:] + bin_edges[:-1]) #needed to plot
    
    bins_index = np.digitize(radius_snapshot, bins=bin_edges) #return the index of the bin for each radius
    
    vel_wrt_com = velocity[time][flag_history[time]]- vel_com_history[time] #- vel_com_history[time]
    rho, phi, vel_r, vel_phi, vel_z = cyl_coord(position[time][flag_history[time]], vel_wrt_com[:, :])

    
    vel_phi_binned = [vel_phi[bins_index==i].tolist() for i in range(1, len(bin_edges)) ] #binning the velocity as the radius
    
    velocity_dispersion = []
    for i in range(len(bin_edges)-1):
        velocity_dispersion.append(np.array(vel_phi_binned[i]).std())
    
    ax.plot(radius_shell, velocity_dispersion, c=c, label=f'Tstep:{time}')
    ax.set_xlabel(r'|$r - r_{com}|_{shell}$ [Henon units]', fontsize=15)
    ax.set_ylabel(r'$\sigma_{{(v- v_{com})}_{\phi, shell}}$ [Henon units]',fontsize=15 )
    ax.set_title('Tangential Velocity dispersion profile across time', fontsize=15)
    ax.set_ylim(-0.3, 5)
    ax.legend()


ax = fig.add_subplot(1,3,3)
for k, c in enumerate(color):
    time = time_to_plot[k]
    position_comFoR_flagged = position_comFoR[time][flag_history[time]]
    radius_snapshot = np.linalg.norm(position_comFoR_flagged, axis=1)
    counts, bin_edges = np.histogram(radius_snapshot, bins=n_bins)
    radius_shell = 1/2 * (bin_edges[1:] + bin_edges[:-1]) #needed to plot
    
    bins_index = np.digitize(radius_snapshot, bins=bin_edges) #return the index of the bin for each radius
    
    vel_wrt_com = velocity[time][flag_history[time]]- vel_com_history[time] #- vel_com_history[time]
    rho, phi, vel_r, vel_phi, vel_z = cyl_coord(position[time][flag_history[time]], vel_wrt_com[:, :])

    
    vel_phi_binned = [vel_r[bins_index==i].tolist() for i in range(1, len(bin_edges)) ] #binning the velocity as the radius
    
    velocity_dispersion = []
    for i in range(len(bin_edges)-1):
        velocity_dispersion.append(np.array(vel_phi_binned[i]).std())
    
    ax.plot(radius_shell, velocity_dispersion, c=c, label=f'Tstep:{time}')
    ax.set_xlabel(r'|$r - r_{com}|_{shell}$ [Henon units]', fontsize=15)
    ax.set_ylabel(r'$\sigma_{{(v- v_{com})}_{r, shell}}$ [Henon units]',fontsize=15 )
    ax.set_title('Radial Velocity dispersion profile across time', fontsize=15)
    ax.set_ylim(-0.3, 5)
    ax.legend()

fig.savefig(fname=f'/ca23/ext_volume/pod_compastro23/Project/Images/{e}/Velocity_dispersion_profile_{N}_InitialPos_{initial_position}_e_{e}.png', bbox_inches='tight')

# Velocity dispersion of the velocity inside the r_h

In [ ]:
fig = plt.figure(figsize=(25, 5))
cmap = mpl.colormaps['turbo']
time_to_plot = np.arange(0, len(position_comFoR), 2)
n_bins=10


velocity_dispersion_half_mass_radius = []
half_mass_radius_list = []
time_list = []

ax = fig.add_subplot(1,3,1)
for k, time in enumerate(time_to_plot):
    position_comFoR_flagged = position_comFoR[time][flag_history[time]]
    radius_snapshot = np.linalg.norm(position_comFoR_flagged, axis=1)
    if np.sort(radius_snapshot).shape[0]<1:
        break
    half_mass_radius = np.sort(radius_snapshot)[int(len(radius_snapshot)/2)]
    
    vel_snapshot = velocity[time][flag_history[time]]-np.linalg.norm(vel_com_history[time]) #- vel_com_history[time]
    vel_modul_snapshot = np.linalg.norm(vel_snapshot, axis=1)
    
    vel_half_mass_radius =  vel_modul_snapshot[np.argsort(radius_snapshot)]
    vel_half_mass_radius =  vel_half_mass_radius[:int(len(vel_half_mass_radius/2))]
    std_half_mass_radius = vel_half_mass_radius.std()
    velocity_dispersion_half_mass_radius.append(std_half_mass_radius)
    half_mass_radius_list.append(half_mass_radius)
    time_list.append(time)
    
pcm = ax.scatter(half_mass_radius_list, velocity_dispersion_half_mass_radius, label=f'Time: {time}', c=time_list, cmap=cmap)
cbar = fig.colorbar(pcm, ax=ax)
cbar.set_label('Time step',  fontsize=10)
ax.set_xlabel('Half mass radius [Henon Unit]', fontsize=15)
ax.set_ylabel(r'$\sigma_{|v - v_{com}|}$ [Henon Unit]',  fontsize=15)
ax.set_title(r'std dev of velocity modulus inside HMR', fontsize=15)


velocity_dispersion_half_mass_radius = []
half_mass_radius_list = []
time_list = []

ax = fig.add_subplot(1,3,2)
for k, time in enumerate(time_to_plot):
    position_comFoR_flagged = position_comFoR[time][flag_history[time]]
    radius_snapshot = np.linalg.norm(position_comFoR_flagged, axis=1)
    if np.sort(radius_snapshot).shape[0]<1:
        break
    half_mass_radius = np.sort(radius_snapshot)[int(len(radius_snapshot)/2)]
    
    vel_wrt_com = velocity[time][flag_history[time]]- vel_com_history[time] #- vel_com_history[time]
    rho, phi, vel_r, vel_phi, vel_z = cyl_coord(position[time][flag_history[time]], vel_wrt_com[:, :])
    
    vel_half_mass_radius =  vel_phi[np.argsort(radius_snapshot)]
    vel_half_mass_radius =  vel_half_mass_radius[:int(len(vel_half_mass_radius/2))]
    std_half_mass_radius = vel_half_mass_radius.std()
    velocity_dispersion_half_mass_radius.append(std_half_mass_radius)
    half_mass_radius_list.append(half_mass_radius)
    time_list.append(time)
    
pcm = ax.scatter(half_mass_radius_list, velocity_dispersion_half_mass_radius, label=f'Time: {time}', c=time_list, cmap=cmap)
cbar = fig.colorbar(pcm, ax=ax)
cbar.set_label('Time step',  fontsize=10)
ax.set_xlabel('Half mass radius [Henon Unit]', fontsize=15)
ax.set_ylabel(r'$\sigma_{{(v - v_{com})}_{\phi}}$ [Henon Unit]', fontsize=15)
ax.set_title(r'std dev of tangential velocity inside HMR', fontsize=15)



velocity_dispersion_half_mass_radius = []
half_mass_radius_list = []
time_list = []

ax = fig.add_subplot(1,3,3)
for k, time in enumerate(time_to_plot):
    position_comFoR_flagged = position_comFoR[time][flag_history[time]]
    radius_snapshot = np.linalg.norm(position_comFoR_flagged, axis=1)
    if np.sort(radius_snapshot).shape[0]<1:
        break
    half_mass_radius = np.sort(radius_snapshot)[int(len(radius_snapshot)/2)]
    
    vel_wrt_com = velocity[time][flag_history[time]]- vel_com_history[time] #- vel_com_history[time]
    rho, phi, vel_r, vel_phi, vel_z = cyl_coord(position[time][flag_history[time]], vel_wrt_com[:, :])
    
    vel_half_mass_radius =  vel_r[np.argsort(radius_snapshot)]
    vel_half_mass_radius =  vel_half_mass_radius[:int(len(vel_half_mass_radius/2))]
    std_half_mass_radius = vel_half_mass_radius.std()
    velocity_dispersion_half_mass_radius.append(std_half_mass_radius)
    half_mass_radius_list.append(half_mass_radius)
    time_list.append(time)
    
pcm = ax.scatter(half_mass_radius_list, velocity_dispersion_half_mass_radius, label=f'Time: {time}', c=time_list, cmap=cmap)
cbar = fig.colorbar(pcm, ax=ax)
cbar.set_label('Time step',  fontsize=10)
ax.set_xlabel('Half mass radius [Henon Unit]',  fontsize=15)
ax.set_ylabel(r'$\sigma_{{(v - v_{com})}_r}$ [Henon Unit]',  fontsize=15)
ax.set_title(r'std dev of radial velocity inside HMR', fontsize=15)
fig.savefig(fname=f'/ca23/ext_volume/pod_compastro23/Project/Images/{e}/Velocity_dispersion_halfmass_{N}_InitialPos_{initial_position}_e_{e}.png', bbox_inches='tight')

# Orbit of the two center of mass (true vs iterative) 

In [ ]:
m = 10
cmap = plt.colormaps['turbo']
cmap2 = plt.colormaps['viridis']
com_history = np.array(com_history)
step = np.arange(0, len(com_history), m)

fig = plt.figure(figsize=(17, 8))
ax = fig.add_subplot(121)
pcm = ax.scatter(position[::m, :, 0].mean(axis=1), position[::m, :, 1].mean(axis=1), c=step,  marker='*', label='center of mass', cmap=cmap)
ax.scatter(com_history[::m, 0], com_history[::m, 1], c=step, marker='.', label='iterative center of mass', cmap=cmap)
ax.scatter(position[0, :, 0].mean(), position[0, :, 1].mean(), marker='*', s=200, label='starting point', color='black')
ax.scatter(0, 0, s=100, label='Point Mass', color='darkgreen', marker='X')
cbar = fig.colorbar(pcm, ax=ax, location='top', aspect=40)
cbar.set_label(r'Time step',fontsize=15)
ax.set_xlim(-semi_major_axis-initial_position, semi_major_axis+5)
ax.set_ylim(-semi_major_axis-5, semi_major_axis+5)
ax.set_xlabel('X [Henon Unit]', fontsize=15)
ax.set_ylabel('Y [Henon Unit]', fontsize=15)
ax.legend(loc='lower left')

ax2 = fig.add_subplot(122)
pcm=ax2.scatter(position[::m, :, 0].mean(axis=1), position[::m, :, 2].mean(axis=1), c=step,  marker='*', label='center of mass', cmap=cmap)
ax2.scatter(com_history[::m, 0], com_history[::m, 2], c=step, marker='.', label='iterative center of mass', cmap=cmap)
ax2.scatter(position[0, :, 0].mean(), position[0, :, 2].mean(), marker='*', s=200, label='starting point', color='black')
cbar = fig.colorbar(pcm, ax=ax2, location='top', aspect=40)
cbar.set_label(r'Time step', fontsize=15)
ax2.scatter(0, 0, s=100, label='Point Mass', color='darkgreen', marker='X')
ax2.set_xlabel('Y [Henon Unit]', fontsize=15)
ax2.set_ylabel('Z [Henon Unit]', fontsize=15)
ax2.set_xlim(-semi_major_axis-initial_position, semi_major_axis+5)
ax2.set_ylim(-semi_major_axis, semi_major_axis)
ax2.legend(loc='lower right')

# Projection on two planes (X vs Y) and (X vs Z)

In [ ]:
important_time = [0, 100, 150, 200, 250, 400]
important_time

## Projection on all the particles

In [ ]:
com_history = np.array(com_history)
step = np.arange(0, len(com_history), 1)

fig = plt.figure(figsize=(25, 15))
for k, time in enumerate(important_time): 
    flag = flag_history[time]
    no_flag = ~flag
    ax = fig.add_subplot(2, 3, k+1)

    ax.scatter(position[time, :, 0][no_flag], position[time, :, 1][no_flag],  marker='.', label=r'$r<2r_t$', zorder=-100, alpha=0.5)
    ax.scatter(position[time, :, 0][flag], position[time, :, 1][flag],  marker='.', label=r'$r<2r_t$', zorder=100, alpha=0.5)
    ax.scatter(com_history[time, 0], com_history[time, 1],  marker='*', s=150, label=f'CoM Tstep={time}', c='blue', zorder=100)
    ax.scatter(0,0, c='darkgreen', marker='X', s=150, label='Point Mass')
    ax.set_xlim(-semi_major_axis-5, semi_major_axis+1)
    ax.set_ylim(-semi_major_axis-5, semi_major_axis+5)
    ax.set_xlabel('X [Henon Unit]', fontsize=15)
    ax.set_ylabel('Y [Henon Unit]', fontsize=15)
    ax.legend()
fig.savefig(fname=f'/ca23/ext_volume/pod_compastro23/Project/Images/{e}/ProjectionXY_{N}_InitialPos_{initial_position}_e_{e}.png', bbox_inches='tight')

# Density plot on X, Y

In [ ]:
def confidence_ellipse(x, y, ax, n_std=3.0, facecolor='none', **kwargs):
    """
    Create a plot of the covariance confidence ellipse of *x* and *y*.

    Parameters
    ----------
    x, y : array-like, shape (n, )
        Input data.

    ax : matplotlib.axes.Axes
        The axes object to draw the ellipse into.

    n_std : float
        The number of standard deviations to determine the ellipse's radiuses.

    **kwargs
        Forwarded to `~matplotlib.patches.Ellipse`

    Returns
    -------
    matplotlib.patches.Ellipse
    """
    if x.size != y.size:
        raise ValueError("x and y must be the same size")

    cov = np.cov(x, y)
    pearson = cov[0, 1]/np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensional dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0), width=ell_radius_x * 2, height=ell_radius_y * 2,
                      facecolor=facecolor, **kwargs)

    # Calculating the standard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std
    mean_x = np.mean(x)

    # calculating the standard deviation of y ...
    scale_y = np.sqrt(cov[1, 1]) * n_std
    mean_y = np.mean(y)

    transf = transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(scale_x, scale_y) \
        .translate(mean_x, mean_y)

    ellipse.set_transform(transf + ax.transData)
    return ax.add_patch(ellipse)

In [ ]:
time_to_plot = [0, 100, 150, 200, 250, 400]
fig=plt.figure(figsize=(25, 10), layout='tight')
for i, time in enumerate(important_time):
    circle = plt.Circle((com_history[time][0], com_history[time][1]), tidal2_history[time], color='black', fill=False, label=r'R< $2r_t$')
    flag=flag_history[time]
    no_flag=~flag
    X = np.stack((position[time, :, 0][no_flag], position[time, :, 1][no_flag], np.zeros_like(position[time, :, 1][no_flag])), axis=1)

    xy = np.vstack([X[:,0], X[:,1]])
    z = gaussian_kde(xy)(xy)
    cov = gaussian_kde(xy).covariance
    

    ax = fig.add_subplot(2, 3, i+1)
    pcm = ax.scatter(X[:, 0], X[:, 1], c=z, cmap=cmap)
    cbar = fig.colorbar(pcm, ax=ax)
    cbar.set_label('Density', fontsize=15)
    ax.scatter(com_history[time, 0], com_history[time, 1],  marker='*', s=150, label=f'CoM Tstep={time}', zorder=100)
    ax.scatter(0, 0, s=100, label='Point Mass', color='darkgreen', marker='X')
    ax.add_patch(circle)
    ax.set_xlim(-semi_major_axis-5, semi_major_axis+5)
    ax.set_ylim(-semi_major_axis-5, semi_major_axis+5)
    ax.set_xlabel('X [Henon Unit]', fontsize=15)
    ax.set_ylabel('Y [Henon Unit]', fontsize=15)
    fig.suptitle("Tidal stream's density", fontsize=25)
    ax.legend()
fig.savefig(fname=f'/ca23/ext_volume/pod_compastro23/Project/Images/{e}/Tidal_stream_density_{N}_InitialPos_{initial_position}_e_{e}.png', bbox_inches='tight')

# Drift of the stars according to their velocity

In [ ]:
time_to_plot = [0, 100, 150, 200, 250, 400]
cmap = plt.colormaps['turbo_r']
com_history = np.array(com_history)
step = np.arange(0, len(com_history), 1)

fig = plt.figure(layout='tight', figsize=(20,10))
for k, time in enumerate(important_time): 
    circle = plt.Circle((com_history[time][0], com_history[time][1]), tidal2_history[time], color='black', fill=False, label=r'R< $2r_t$')

    # vel_rho, vel_phi = cyl_coord(velocity[time, :, :])
    # vel_rho_com, vel_phi_com = cyl_coord(vel_com_history[time])  

    rho, phi, vel_r, vel_phi, vel_z = cyl_coord(position[time, :, :], velocity[time, :, :])
    rho_com, phi_com, vel_r_com, vel_phi_com, vel_z_com = cyl_coord(com_history[time], vel_com_history[time])
    vel_wrt_com = vel_phi - vel_phi_com
    ax = fig.add_subplot(2, 3, k+1)

    ax.scatter(0,0, c='darkgreen', marker='X', label='Point Mass', s=150)
    ax.scatter(com_history[time, 0], com_history[time, 1],  marker='*', s=150, label=f'CoM Tstep={time}', zorder=100)
    pcm = ax.scatter(position[time, :, 0], position[time, :, 1], c=vel_wrt_com/vel_wrt_com.max(), marker='.', cmap=cmap)
    ax.add_patch(circle)
    cbar = fig.colorbar(pcm, ax=ax)
    cbar.set_label(r'$\frac{v_\phi - v_{\phi, com}}{v_{\phi,max}}$', fontsize=15)
    ax.set_xlim(-semi_major_axis-5, semi_major_axis+5)
    ax.set_ylim(-semi_major_axis-5, semi_major_axis+5)
    ax.set_xlabel('X [Henon Unit]', fontsize=15)
    ax.set_ylabel('Y [Henon Unit]', fontsize=15)
    ax.legend()
fig.savefig(fname=f'/ca23/ext_volume/pod_compastro23/Project/Images/{e}/TanVel_{N}_InitialPos_{initial_position}_e_{e}.png', bbox_inches='tight')

In [ ]:
cmap = plt.colormaps['turbo']
com_history = np.array(com_history)
step = np.arange(0, len(com_history), 1)

fig = plt.figure(layout='tight', figsize=(20,10))
for k, time in enumerate(important_time): 
    circle = plt.Circle((com_history[time][0], com_history[time][1]), tidal2_history[time], color='r', fill=False, label=r'R< $2r_t$')
    
    rho, phi, vel_r, vel_phi, vel_z = cyl_coord(position[time, :, :], velocity[time, :, :])
    rho_com, phi_com, vel_r_com, vel_phi_com, vel_z_com = cyl_coord(com_history[time], vel_com_history[time])  
    vel_phi_wrt_com = vel_phi - vel_phi_com
    vel_rho_wrt_com = vel_r - vel_r_com
    E_kin = 0.5*(1/N)*np.linalg.norm(velocity[time, :, :], axis=1)**2
    E_pot = potential_energy[time] + (1/N)*M_G/(np.linalg.norm(position[time], axis=1))
    E_tot = E_kin + E_pot
    ax = fig.add_subplot(2, 3, k+1)

    pcm = ax.scatter(position[time, :, 0][np.linalg.norm(position[time], axis=1)<semi_major_axis+10], position[time, :, 1][np.linalg.norm(position[time], axis=1)<semi_major_axis+10],  c=E_tot[np.linalg.norm(position[time], axis=1)<semi_major_axis+10],  marker='.', cmap=cmap)
    ax.scatter(com_history[time][0], com_history[time][1], marker='*', s=150, c='r', label=f'Com Tstep:{time}')
    ax.scatter(0,0, marker='X', color='darkgreen', label='Poit Mass', s=150)
    ax.add_patch(circle)
    
    cbar = fig.colorbar(pcm, ax=ax)
    cbar.set_label(r'$E_{tot} - U_{Point Mass}$', fontsize=15)
    
    ax.set_xlabel('X [Henon Unit]', fontsize=15)
    ax.set_ylabel('Y [Henon Unit]', fontsize=15)
    # ax.set_xlim(com_history[time][0]-10,  com_history[time][0]+10)
    # ax.set_ylim(com_history[time][1]-10,  com_history[time][1]+10)
    ax.legend()
    fig.show()
fig.savefig(fname=f'/ca23/ext_volume/pod_compastro23/Project/Images/{e}/EtotPlummer_{N}_InitialPos_{initial_position}_e_{e}.png', bbox_inches='tight')

# Histogram 

In [ ]:
fig = plt.figure(figsize=(20, 30), layout='tight')
k=1
for time in important_time:
    cmap = plt.colormaps['turbo_r']
    flag=flag_history[time]
    no_flag=~flag
    
    
    rho_com, phi_com, vel_r_com, vel_phi_com, vel_z_com = cyl_coord(com_history[time], vel_com_history[time])
    rho, phi, vel_r, vel_phi, vel_z = cyl_coord(position[time][no_flag], velocity[time][no_flag])
    vel_wrt_com = vel_phi - vel_phi_com
    
    plummer_shpere, plummer_shpere_velocity = position[time][flag], velocity[time][flag]
    rho_plummer, phi_plummer, vel_r_plummer, vel_phi_plummer, vel_z_plummer = cyl_coord(plummer_shpere, plummer_shpere_velocity)
    vel_wrt_com_plummer = vel_phi_plummer - vel_phi_com
    
    
    ax = fig.add_subplot(len(important_time), 2, k)
    ax.scatter(0,0, label='Point Mass', s=100, color='darkgreen',  marker='X')
    pcm = ax.scatter(position[time][no_flag][:, 0], position[time][no_flag][:, 1],  c=vel_wrt_com/vel_wrt_com.max(),  marker='.', cmap=cmap)
    ax.scatter(com_history[time][0], com_history[time][1], marker='*', s=100, label=f'CoM, Time:{time}')
    cbar = fig.colorbar(pcm, ax=ax)
    cbar.set_label(r'$\frac{v_\phi - v_{\phi, com}}{v_{\phi,max}}$',  fontsize=15)
    # pcm.set_clim(vmin=-1.0, vmax=1.0)
    ax.set_xlabel('X [Henon Unit]', fontsize=15)
    ax.set_ylabel('Y [Henon Unit]', fontsize=15)
    ax.set_xlim(-semi_major_axis-5, semi_major_axis+5)
    ax.set_ylim(-semi_major_axis-5, semi_major_axis+5)
    ax.legend()
    k+=1
    
    
    
    ax = fig.add_subplot(len(important_time), 2, k)
    ax.hist(vel_wrt_com[phi-phi_com>0]/vel_wrt_com.max(), bins= 'sqrt', density=True, color='tab:blue', alpha=0.3)
    ax.hist(vel_wrt_com[phi-phi_com<0]/vel_wrt_com.max(), bins='sqrt', density=True,color='tab:red', alpha=0.3)
    ax.hist(vel_wrt_com_plummer/vel_wrt_com.max(), bins='sqrt', density=True, color='lawngreen', alpha=0.3)
    
    density_high = gaussian_kde(vel_wrt_com[(phi-phi_com)>0]/vel_wrt_com.max())
    density_low = gaussian_kde(vel_wrt_com[(phi-phi_com)<0]/vel_wrt_com.max())
    density_plummer = gaussian_kde(vel_wrt_com_plummer/vel_wrt_com.max())
    x = np.linspace(vel_wrt_com.min(), vel_wrt_com.max(), 1000)
    ax.plot(x, density_high(x), label=r'$\phi>\phi_{com}$', color='tab:blue')
    ax.plot(x, density_low(x), label=r'$\phi<\phi_{com}$', color='tab:red')
    ax.plot(x, density_plummer(x), label=r'$r < 2r_t$', color='tab:green')
    
    ax.set_xlabel(r'$\frac{v_\phi - v_{\phi, com}}{v_{\phi,max}}$', fontsize=15)
    ax.set_ylabel('density',  fontsize=15)
    ax.yaxis.set_label_position("right")
    ax.yaxis.tick_right()
    
    ax.set_xlim(-1, 1)
    ax.set_ylim(-0.1, 10)
    ax.legend()
    k+=1
fig.savefig(fname=f'/ca23/ext_volume/pod_compastro23/Project/Images/{e}/HistTidalVel_{N}_InitialPos_{initial_position}_e_{e}.png', bbox_inches='tight')